In [1]:
import os
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
sc=SparkContext()
sqlContext=SQLContext(sc)
import urllib
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType ,DoubleType ,BooleanType, IntegerType
from pyspark.sql.functions import udf ,col ,upper
import numpy

In [2]:
titanic_train = sqlContext.read.option("header","true").csv('file:///home/garth/titanic/train.csv',inferSchema='true')

In [3]:
titanic_train.show(5,truncate=False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |null |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |null |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|35.0|1    |0    |113803          |53

In [4]:
sqlContext.registerDataFrameAsTable(titanic_train, "titanic_table")

In [5]:
#sqlContext.sql("select count(*) from titanic_table where Survived=1 and Cabin like '%F%' ").show()
#sqlContext.sql("select count(*) from titanic_table where Survived=0 and Cabin like '%F%'").show()
sqlContext.sql("select count(*) from titanic_table where Survived=0 and Cabin =''").show()
sqlContext.sql("select count(*) from titanic_table where Survived=1 and Cabin !=''").show()
print "Impute Sex?"
sqlContext.sql("select count(*) from titanic_table where Sex!='male' and Sex!='female'").show()

print "male ratio"
print float(titanic_train.filter(titanic_train.Sex=="male").count())/(titanic_train.count())

+--------+
|count(1)|
+--------+
|       0|
+--------+

+--------+
|count(1)|
+--------+
|     136|
+--------+

Impute Sex?
+--------+
|count(1)|
+--------+
|       0|
+--------+

male ratio
0.64758698092


In [215]:
def isMr(name):
    if "Mr." in name:
        return 1
    else:
        return 0

isMr_udf=udf(lambda x: isMr(x),IntegerType())

def isMrs(name):
    if "Mrs." in name or "Countess" in name or "Mme" in name or "Lady" in name:
        return 1
    else:
        return 0

isMrs_udf=udf(lambda x: isMrs(x),IntegerType())

def isMiss(name):
    if "Miss." in name or ("Mlle" in name or "Ms" in name):
        return 1
    else:
        return 0

isMiss_udf=udf(lambda x: isMiss(x),IntegerType())

def isMaster(name):
    if "Master." in name:
        return 1
    else:
        return 0

isMaster_udf=udf(lambda x: isMaster(x),IntegerType())

def isRoyal(name):
    if "Jonkheer." in name or "Countess." in name or "Count" in name or "Jonkvrouw." in name :
        return 1
    else:
        return 0

isRoyal_udf=udf(lambda x: isRoyal(x),IntegerType())

def isArmy(name):
    if "Major." in name or "Col." in name or "Capt." in name :
        return 1
    else:
        return 0

isArmy_udf=udf(lambda x: isArmy(x),IntegerType())

def isRev(name):
    if "Rev." in name :
        return 1
    else:
        return 0

isRev_udf=udf(lambda x: isRev(x),IntegerType())

def isDr(name):
    if "Dr." in name :
        return 1
    else:
        return 0

isDr_udf=udf(lambda x: isDr(x),IntegerType())

def hasFam(a,b):
    if a!=0 or b!=0:
        return 1
    else:
        return 0

hasFam_udf=udf(lambda x,y:hasFam(x,y),IntegerType())

def getSexasNum(a):
    if a=="male":
        return 1
    else:
        return 0

sexAsNum_udf=udf(lambda x:getSexasNum(x),IntegerType())

def embQ(a):
    if a=="Q":
        return 1
    else:
        return 0

embQ_udf=udf(lambda x:embQ(x),IntegerType())

def embS(a):
    if a=="S":
        return 1
    else:
        return 0

embS_udf=udf(lambda x:embS(x),IntegerType())

def embC(a):
    if a=="C":
        return 1
    else:
        return 0

embC_udf=udf(lambda x:embC(x),IntegerType())

def fixAge(age,Name,pclass):
    if age == None and ("Mr." in Name) and (pclass==1):
        return 41.58
    elif age == None and ("Mrs." in Name) and (pclass==1):
        return 40.88
    elif age == None and ("Master." in Name) and (pclass==1):
        return 5.3
    elif age == None and ("Miss." in Name) and (pclass==1):
        return 30.0
    elif age == None and ("Dr." in Name) and (pclass==1):
        return 43.75
    elif age == None and ("Rev." in Name) and (pclass==1):
        return 43.0
    elif age == None and ("Major" in Name) and (pclass==1):
        return 48.5
    elif age == None and ("Capt." in Name) and (pclass==1):
        return 70.0
    elif age == None and ("Col" in Name) and (pclass==1):
        return 58.0
    
    elif age == None and ("Mr." in Name) and (pclass==2):
        return 32.77
    elif age == None and ("Mrs." in Name) and (pclass==2):
        return 33.68
    elif age == None and ("Master." in Name) and (pclass==2):
        return 2.25
    elif age == None and ("Miss." in Name) and (pclass==2):
        return 22.39
    elif age == None and ("Dr." in Name) and (pclass==2):
        return 38.5
    elif age == None and ("Rev." in Name) and (pclass==2):
        return 43.0
    elif age == None and ("Major" in Name) and (pclass==2):
        return 48.5
    elif age == None and ("Capt." in Name) and (pclass==2):
        return 70.0
    elif age == None and ("Col" in Name) and (pclass==2):
        return 58.0
    
    elif age == None and ("Mr." in Name) and (pclass==3):
        return 28.72
    elif age == None and ("Mrs." in Name) and (pclass==3):
        return 33.51
    elif age == None and ("Master." in Name) and (pclass==3):
        return 5.35
    elif age == None and ("Miss." in Name) and (pclass==3):
        return 16.12
    elif age == None and ("Dr." in Name) and (pclass==3):
        return 38.5
    elif age == None and ("Rev." in Name) and (pclass==3):
        return 43.0
    elif age == None and ("Major" in Name) and (pclass==3):
        return 48.5
    elif age == None and ("Capt." in Name) and (pclass==3):
        return 70.0
    elif age == None and ("Col" in Name) and (pclass==3):
        return 58.0
    
    elif age == None:
        return 30.0
    
    else:
        return float(age)

fixAge_udf=udf(lambda x,y,z:fixAge(x,y,z),DoubleType())

def fixFare(Fare,pclass):
    if (pclass==1 ) and (Fare=='0.0' or (Fare == "null")):
        return 84.15
    elif (pclass==2) and (Fare=='0.0' or (Fare == "null")):
        return 20.61
    elif (pclass==3) and (Fare=='0.0' or (Fare == "null")):
        return 13.70
    else:
        return Fare

fixFare_udf=udf(lambda x,y:fixFare(x,y),DoubleType())



def relativeFare(Fare,pclass):
    if (pclass==1) and (Fare is not None):
        return (Fare-84.15) 
    elif (pclass==2) and (Fare is not None):
        return (Fare-20.61)
    elif (pclass==3) and (Fare is not None):
        return (Fare-13.70)
    else:
        return 0.0
    
relativeFare_udf=udf(lambda x,y:relativeFare(x,y),DoubleType())

def isWife(SibSp,SexAsNum,Age):
    if (SibSp>0.0) and (SexAsNum==0) and (Age>=18.0):
        return 1.0
    else:
        return 0.0
    
isWife_udf=udf(lambda x,y,z:isWife(x,y,z),DoubleType())


def isHusband(SibSp,SexAsNum,Age):
    if (SibSp>0.0) and (SexAsNum==1) and (Age>=18.0):
        return 1.0
    else:
        return 0.0
    
isHusband_udf=udf(lambda x,y,z:isHusband(x,y,z),DoubleType())


def isParent(Parch,Age):
    if (Parch>0.0) and (Age>18.0):
        return 1.0
    else:
        return 0.0
    
isParent_udf=udf(lambda x,y:isParent(x,y),DoubleType())


def hasParents(Parch,Age):
    if (Parch>0.0) and (Age<18.0):
        return 1.0
    else:
        return 0.0
    
hasParents_udf=udf(lambda x,y:hasParents(x,y),DoubleType())

def getTicketNum(Ticket):
    
    if Ticket is None:
        return 0.0
    else:
        arr=Ticket.split(' ')
    
    if (len(arr) == 1):
        try:
            return float(arr[0])
        except ValueError:
            return 0.0
    elif (len(arr)==2):
        try:
            return float(arr[1])
        except ValueError:
            return 0.0
    else:
        return 0.0 
    
getTicketNum_udf=udf(lambda y:getTicketNum(y),DoubleType())

def getTicketPrefix(Ticket):
    
    if Ticket is None:
        return "none"
    else:
        arr=Ticket.split(' ')
    
    if (len(arr) == 1):
        try:
            return "none"
        except ValueError:
            return "none"
    elif (len(arr)==2):
        try:
            return (arr[0])
        except ValueError:
            return "none"
    else:
        return "none" 
    
getTicketPrefix_udf=udf(lambda y:getTicketPrefix(y),StringType())

def getDeck(Cabin):
    
    if Cabin is None:
        return 0.0
    elif ("A" in Cabin):
        return 1.0
    elif ("B" in Cabin):
        return 2.0
    elif ("C" in Cabin):
        return 3.0
    elif ("D" in Cabin):
        return 4.0
    elif ("E" in Cabin):
        return 5.0
    elif ("F" in Cabin):
        return 6.0
    else:
        return 0.0 
    
getDeck_udf=udf(lambda y:getDeck(y),DoubleType())


def isCabA(Cabin):
    if Cabin is None:
        return 0.0
    elif ("A" in Cabin):
        return 1.0
    else:
        return 0.0
    
isCabA_udf=udf(lambda y:isCabA(y),DoubleType())


def isCabB(Cabin):
    if Cabin is None:
        return 0.0
    elif ("B" in Cabin):
        return 1.0
    else:
        return 0.0
    
isCabB_udf=udf(lambda y:isCabB(y),DoubleType())


def isCabC(Cabin):
    if Cabin is None:
        return 0.0
    elif ("C" in Cabin):
        return 1.0
    else:
        return 0.0
    
isCabC_udf=udf(lambda y:isCabC(y),DoubleType())


def isCabD(Cabin):
    if Cabin is None:
        return 0.0
    elif ("D" in Cabin):
        return 1.0
    else:
        return 0.0
    
isCabD_udf=udf(lambda y:isCabD(y),DoubleType())


def isCabE(Cabin):
    if Cabin is None:
        return 0.0
    elif ("E" in Cabin):
        return 1.0
    else:
        return 0.0
    
isCabE_udf=udf(lambda y:isCabE(y),DoubleType())

def isCabF(Cabin):
    if Cabin is None:
        return 0.0
    elif ("F" in Cabin):
        return 1.0
    else:
        return 0.0
    
isCabF_udf=udf(lambda y:isCabF(y),DoubleType())

def hasCab(Cabin):
    if Cabin is None:
        return 0.0
    elif (Cabin==''):
        return 0.0
    else:
        return 1.0
    
hasCab_udf=udf(lambda y:hasCab(y),DoubleType())

def nameLength(Name):
    if Name is None:
        return 0.0
    elif (Name==''):
        return 0.0
    else:
        return float(len(Name))
    
nameLength_udf=udf(lambda y:nameLength(y),DoubleType())

In [216]:
features_df=titanic_train.withColumn("isRev",isRev_udf(titanic_train.Name))
features_df=  features_df.withColumn("isMr", isMr_udf(features_df.Name))
features_df=  features_df.withColumn("isMrs",isMrs_udf(features_df.Name))
features_df=  features_df.withColumn("isDr", isDr_udf(features_df.Name))
features_df=  features_df.withColumn("isMaster",isMaster_udf(features_df.Name))
features_df=  features_df.withColumn("isMiss",isMiss_udf(features_df.Name))
features_df=  features_df.withColumn("isRev",isRev_udf(features_df.Name))
features_df=  features_df.withColumn("isArmy",isArmy_udf(features_df.Name))
features_df=  features_df.withColumn("isRoyal",isRoyal_udf(features_df.Name))   #.drop("Name")

features_df=  features_df.withColumn("embQ",embQ_udf(features_df.Embarked))
features_df=  features_df.withColumn("embS",embS_udf(features_df.Embarked))
features_df=  features_df.withColumn("embC",embC_udf(features_df.Embarked)).drop("Embarked")

features_df=  features_df.withColumn("hasFam",hasFam_udf(features_df.Parch,features_df.SibSp))

features_df=  features_df.withColumn("sexAsNum",sexAsNum_udf(features_df.Sex)).drop("Sex")

features_df=  features_df.withColumn("Age2",fixAge_udf(features_df.Age,features_df.Name,features_df.Pclass))

features_df= features_df.withColumn("Fare2",fixFare_udf(features_df.Fare,features_df.Pclass))

features_df= features_df.withColumn("FareRelative",relativeFare_udf(features_df.Fare2,features_df.Pclass))

features_df= features_df.withColumn("isWife",isWife_udf(features_df.SibSp,features_df.sexAsNum,features_df.Age2))
features_df= features_df.withColumn("isHusband",isHusband_udf(features_df.SibSp,features_df.sexAsNum,features_df.Age2))
features_df= features_df.withColumn("isParent",isParent_udf(features_df.Parch,features_df.Age2))
features_df= features_df.withColumn("hasParents",hasParents_udf(features_df.Parch,features_df.Age2))

features_df= features_df.withColumn("TicketNumber",getTicketNum_udf(features_df.Ticket))
features_df= features_df.withColumn("TicketPrefix",getTicketPrefix_udf(features_df.Ticket))

features_df=  features_df.withColumn("cabA",isCabA_udf(features_df.Cabin))
features_df=  features_df.withColumn("cabB",isCabB_udf(features_df.Cabin))
features_df=  features_df.withColumn("cabC",isCabC_udf(features_df.Cabin))
features_df=  features_df.withColumn("cabD",isCabD_udf(features_df.Cabin))
features_df=  features_df.withColumn("cabE",isCabE_udf(features_df.Cabin))
features_df=  features_df.withColumn("cabF",isCabF_udf(features_df.Cabin))

features_df=  features_df.withColumn("hasCab",hasCab_udf(features_df.Cabin))

features_df= features_df.withColumn("Deck",getDeck_udf(features_df.Cabin))

features_df= features_df.withColumn("nameLength",nameLength_udf(features_df.Name))


#features_df= features_df.drop("Ticket").drop("Cabin").drop("Name").drop("Age").drop("Fare").drop("SibSp").drop("Parch")
features_df= features_df.drop("Ticket").drop("Name").drop("Age").drop("Fare").drop("SibSp").drop("Parch")

In [266]:
#features_df.show(30)
#features_df.describe().show()

#sqlContext.registerDataFrameAsTable(features_df,"features_df")

#sqlContext.sql("SELECT avg(Fare) from features_df where Pclass=1").show()
#sqlContext.sql("SELECT avg(Fare) from features_df where Pclass=2").show()
#sqlContext.sql("SELECT avg(Fare) from features_df where Pclass=3").show()

#sqlContext.sql("SELECT avg(Fare2) from features_df where embC=1").show()
#sqlContext.sql("SELECT avg(Fare2) from features_df where embQ=1").show()
#sqlContext.sql("SELECT avg(Fare2) from features_df where embS=1").show()

#sqlContext.sql("SELECT avg(Fare2) from features_df where Pclass=1 and embC=1").show()
#sqlContext.sql("SELECT count(*) from features_df where Pclass=1 and embQ=1").show()
#sqlContext.sql("SELECT avg(Fare2) from features_df where Pclass=1 and embS=1").show()

#sqlContext.sql("SELECT count(*) from features_df").show()
#sqlContext.sql("SELECT count( distinct TicketPrefix) from features_df").show()
#sqlContext.sql("SELECT distinct TicketPrefix from features_df").show()
#sqlContext.sql("SELECT count(*) from features_df where TicketPrefix = 'none'").show()
#sqlContext.sql("SELECT count(*) from features_df where TicketPrefix != 'none'").show()
#sqlContext.sql("SELECT distinct(FareRelative) from features_df ").show()
#sqlContext.sql("SELECT count(distinct Cabin) from features_df ").show()

In [176]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest

In [219]:
#features_df.show(truncate=False)

In [292]:
#sqlContext=SQLContext(sc)
lpoints=features_df.rdd.map(lambda x: LabeledPoint(x.Survived,[
                                                                      x.sexAsNum
                                                                     ,x.Pclass
                                                                     ,x.Age2
                                                                     ,x.Fare2
                                                                     ,x.isRev
                                                                     ,x.isMr
                                                                     ,x.isMrs
                                                                     ,x.isDr
                                                                     ,x.isMaster
                                                                     ,x.isMiss
                                                                     ,x.isArmy
                                                                     ,x.isRoyal
                                                                     ,x.embQ
                                                                     ,x.embS
                                                                     ,x.embC
                                                                     ,x.hasFam
                                                                     ,x.FareRelative
                                                                     ,x.hasParents
                                                                     ,x.isParent
                                                                     ,x.isHusband
                                                                     ,x.isWife                
                                                                     ,x.TicketNumber
                                                                     ,x.Deck
                                                     #                ,x.cabA
                                                     #                ,x.cabB
                                                     #                ,x.cabC
                                                     #                ,x.cabD
                                                     #                ,x.cabE
                                                     #                ,x.cabF
                                                                      ,x.hasCab
                                                                      ,x.nameLength
                                                                    ]))
(trainingData, testData) = lpoints.randomSplit([0.8, 0.2])
#(trainingData2, testData2) = lpoints.randomSplit([0.8, 0.2])
#(testData,trainingData, testData2) = lpoints.randomSplit([0.15, 0.7,0.15])


In [293]:
#model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},impurity='entropy', maxDepth=30, maxBins=128)
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=256, featureSubsetStrategy="auto",
                                     impurity='entropy', maxDepth=30, maxBins=64)


In [294]:
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Error = ' + str(testErr))
print('Accuracy = ' + str(1-testErr))

Error = 0.174863387978
Accuracy = 0.825136612022


In [285]:
#predictions2 = model.predict(testData2.map(lambda x: x.features))
#labelsAndPredictions2 = testData2.map(lambda lp: lp.label).zip(predictions2)
#testErr = labelsAndPredictions2.filter(lambda (v, p): v != p).count() / float(testData2.count())
#print('Test Error = ' + str(testErr))

In [286]:
titanic_test = sqlContext.read.option("header","true").csv('file:///home/garth/titanic/test.csv',inferSchema='true')
titanic_test=  titanic_test.withColumn("isRev",isRev_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isMr", isMr_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isMrs",isMrs_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isDr", isDr_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isMaster",isMaster_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isMiss",isMiss_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isRev",isRev_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isArmy",isArmy_udf(titanic_test.Name))
titanic_test=  titanic_test.withColumn("isRoyal",isRoyal_udf(titanic_test.Name))   #.drop("Name")

titanic_test=  titanic_test.withColumn("embQ",embQ_udf(titanic_test.Embarked))
titanic_test=  titanic_test.withColumn("embS",embS_udf(titanic_test.Embarked))
titanic_test=  titanic_test.withColumn("embC",embC_udf(titanic_test.Embarked)).drop("Embarked")

titanic_test=  titanic_test.withColumn("hasFam",hasFam_udf(titanic_test.Parch,titanic_test.SibSp))

titanic_test=  titanic_test.withColumn("sexAsNum",sexAsNum_udf(titanic_test.Sex)).drop("Sex")

titanic_test=  titanic_test.withColumn("Age2",fixAge_udf(titanic_test.Age,titanic_test.Name,titanic_test.Pclass))

titanic_test=  titanic_test.withColumn("Fare2",fixFare_udf(titanic_test.Fare,titanic_test.Pclass))#.drop("Fare")

titanic_test=  titanic_test.withColumn("FareRelative",relativeFare_udf(titanic_test.Fare2,titanic_test.Pclass))

titanic_test= titanic_test.withColumn("isWife",isWife_udf(titanic_test.SibSp,titanic_test.sexAsNum,titanic_test.Age2))
titanic_test= titanic_test.withColumn("isHusband",isHusband_udf(titanic_test.SibSp,titanic_test.sexAsNum,titanic_test.Age2))
titanic_test= titanic_test.withColumn("isParent",isParent_udf(titanic_test.Parch,titanic_test.Age2))
titanic_test= titanic_test.withColumn("hasParents",hasParents_udf(titanic_test.Parch,titanic_test.Age2))

titanic_test= titanic_test.withColumn("TicketNumber",getTicketNum_udf(titanic_test.Ticket))
titanic_test= titanic_test.withColumn("TicketPrefix",getTicketPrefix_udf(titanic_test.Ticket))

titanic_test= titanic_test.withColumn("Deck",getDeck_udf(titanic_test.Cabin))

titanic_test=  titanic_test.withColumn("cabA",isCabA_udf(titanic_test.Cabin))
titanic_test=  titanic_test.withColumn("cabB",isCabB_udf(titanic_test.Cabin))
titanic_test=  titanic_test.withColumn("cabC",isCabC_udf(titanic_test.Cabin))
titanic_test=  titanic_test.withColumn("cabD",isCabD_udf(titanic_test.Cabin))
titanic_test=  titanic_test.withColumn("cabE",isCabE_udf(titanic_test.Cabin))
titanic_test=  titanic_test.withColumn("cabF",isCabF_udf(titanic_test.Cabin))

titanic_test=  titanic_test.withColumn("hasCab",hasCab_udf(titanic_test.Cabin))

titanic_test= titanic_test.withColumn("nameLength",nameLength_udf(titanic_test.Name))

titanic_test=  titanic_test.drop("Ticket").drop("Cabin").drop("Name").drop("Age").drop("SibSp").drop("Parch")

titanic_test.show(1)

+-----------+------+------+-----+----+-----+----+--------+------+------+-------+----+----+----+------+--------+----+------+------------------+------+---------+--------+----------+------------+------------+----+----+----+----+----+----+----+------+----------+
|PassengerId|Pclass|  Fare|isRev|isMr|isMrs|isDr|isMaster|isMiss|isArmy|isRoyal|embQ|embS|embC|hasFam|sexAsNum|Age2| Fare2|      FareRelative|isWife|isHusband|isParent|hasParents|TicketNumber|TicketPrefix|Deck|cabA|cabB|cabC|cabD|cabE|cabF|hasCab|nameLength|
+-----------+------+------+-----+----+-----+----+--------+------+------+-------+----+----+----+------+--------+----+------+------------------+------+---------+--------+----------+------------+------------+----+----+----+----+----+----+----+------+----------+
|        892|     3|7.8292|    0|   1|    0|   0|       0|     0|     0|      0|   1|   0|   0|     0|       1|34.5|7.8292|-5.870799999999999|   0.0|      0.0|     0.0|       0.0|    330911.0|        none| 0.0| 0.0| 0.0| 0.

In [287]:
sqlContext.registerDataFrameAsTable(titanic_test,"titanic_test")
sqlContext.sql("SELECT Fare,Fare2 from titanic_test where Fare2 is null").show()

+----+-----+
|Fare|Fare2|
+----+-----+
|null| null|
+----+-----+



In [288]:
lpointsTest=titanic_test.rdd.map(lambda x: LabeledPoint(0,[
                                                                      x.sexAsNum
                                                                     ,x.Pclass
                                                                     ,x.Age2
                                                                     ,x.Fare2
                                                                     ,x.isRev
                                                                     ,x.isMr
                                                                     ,x.isMrs
                                                                     ,x.isDr
                                                                     ,x.isMaster
                                                                     ,x.isMiss
                                                                     ,x.isArmy
                                                                     ,x.isRoyal
                                                                     ,x.embQ
                                                                     ,x.embS
                                                                     ,x.embC
                                                                     ,x.hasFam
                                                                     ,x.FareRelative
                                                                     ,x.hasParents
                                                                     ,x.isParent
                                                                     ,x.isHusband
                                                                     ,x.isWife
                                                                     ,x.TicketNumber
                                                                     ,x.Deck
                                                             #        ,x.cabA
                                                             #        ,x.cabB
                                                             #        ,x.cabC
                                                             #        ,x.cabD
                                                             #        ,x.cabE
                                                             #        ,x.cabF
                                                                     ,x.hasCab
                                                                     ,x.nameLength
                                                                    ]))


In [295]:
unseen_predictions=model.predict(lpointsTest.map(lambda x:x.features))

predictions = map(int, unseen_predictions.collect())

In [296]:
passengerId= titanic_test.rdd.map(lambda x:x.PassengerId).collect()
answers=(zip(passengerId,predictions))

In [297]:
outfile = open("/home/garth/titanic_answer.csv", "w")
#print "PassengerId,Surived" >> outfile
print >> outfile, str("PassengerId,Survived")
print >> outfile, "\n".join(str(i).replace("(","").replace(")","").replace(" ","") for i in answers)
outfile.close()

In [230]:
#titanic_train.drop("Name").drop("Cabin").drop("Ticket").drop("Passengerid").drop("SibSp").drop("Pclass").describe().show()

In [231]:
#print(model.toDebugString())

In [232]:
import matplotlib.pyplot as plt
plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

In [233]:
people = ('Tom', 'Dick', 'Harry', 'Slim', 'Jim')
y_pos = np.arange(len(people))
performance = 3 + 10 * np.random.rand(len(people))
error = np.random.rand(len(people))

plt.barh(y_pos, performance, xerr=error, align='center', alpha=0.4)
plt.yticks(y_pos, people)
plt.xlabel('Performance')
plt.title('How fast do you want to go today?')

plt.show()

# XGBoosted Trees

In [234]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.linalg import Vectors 
from pyspark.ml.linalg import VectorUDT


In [298]:
data=features_df.rdd.map(lambda x: (float(x.Survived),Vectors.dense([
                                                                      x.sexAsNum
                                                                     ,x.Pclass
                                                                     ,x.Age2
                                                                     ,x.Fare2
                                                                     ,x.isRev
                                                                     ,x.isMr
                                                                     ,x.isMrs
                                                                     ,x.isDr
                                                                     ,x.isMaster
                                                                     ,x.isMiss
                                                                     ,x.isArmy
                                                                     ,x.isRoyal
                                                                     ,x.embQ
                                                                     ,x.embS
                                                                     ,x.embC
                                                                     ,x.hasFam
                                                                     ,x.FareRelative
                                                                     ,x.hasParents
                                                                     ,x.isParent
                                                                     ,x.isHusband
                                                                     ,x.isWife                
                                                                     ,x.TicketNumber
                                                                     ,x.Deck
                                                                     ,x.cabA
                                                                     ,x.cabB
                                                                     ,x.cabC
                                                                     ,x.cabD
                                                                     ,x.cabE
                                                                     ,x.cabF
                                                                     ,x.hasCab
                                                                     ,x.nameLength
                                                                    ]))).toDF(["label","features"])

In [299]:
#data.show(1,truncate=False)
#print data.count()

In [315]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10).fit(data)
(trainingData, testData) = data.randomSplit([0.8, 0.2])
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])
model = pipeline.fit(trainingData)
predictions = model.transform(testData)
#predictions.select("prediction", "indexedLabel", "features").show(1,truncate=False)
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
gbtModel = model.stages[2]
print(gbtModel)
#print model.stages

Test Error = 0.19209
GBTClassificationModel (uid=GBTClassifier_405990ef6805b465d5da) with 10 trees


In [ ]:
def avePrediction(x):
    sumErr = 0
    for i in range(0, x):
        labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
        featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10).fit(data)
        (trainingData, testData) = data.randomSplit([0.8, 0.2])
        gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)
        pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])
        model = pipeline.fit(trainingData)
        predictions = model.transform(testData)
        evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
        accuracy = evaluator.evaluate(predictions)
        print("Test Error = %g" % (1.0 - accuracy))
        gbtModel = model.stages[2]
        sumErr = sumErr+(1.0-accuracy)
    return (sumErr/x)

x = avePrediction(10)
print x

In [203]:
sqlContext.registerDataFrameAsTable(predictions,"predictions")
truePos =sqlContext.sql("SELECT count(*) from predictions where label='1' and prediction='1' ").first()['count(1)']
falsePos=sqlContext.sql("SELECT count(*) from predictions where label='1' and prediction='0' ").first()['count(1)']
falseNeg=sqlContext.sql("SELECT count(*) from predictions where label='0' and prediction='1' ").first()['count(1)']
trueNeg =sqlContext.sql("SELECT count(*) from predictions where label='0' and prediction='0' ").first()['count(1)']

print "true Positives:"
print truePos

print "true Negatives:"
print trueNeg

print "False Positives:"
print falsePos

print "False Negatives:"
print falseNeg



true Positives:
49
true Negatives:
97
False Positives:
11
False Negatives:
23


## Compare Test and Training

In [310]:
#numMales= features_df.filter(features_df.sexAsNum==1).count()
#numFemales= features_df.filter(features_df.sexAsNum==0).count()

print "total size of training set"
print features_df.count()

print "total size of testData (subsection of training)"
print testData.count()

numMales=testData.rdd.map(lambda x:x[0]).reduce(lambda x,y:x+y)
print numMales
numFemales=testData.count()-numMales
print numFemales
print "ratio of males in testData:"
print (float(numMales)/(numFemales+numMales))
#print float(float(numMales)/(numMales+numFemales))

print "ratio of males in training:"
print (float(trainingData.rdd.map(lambda x:x[0]).reduce(lambda x,y:x+y))/(trainingData.count()))

total size of training set
891
total size of testData (subsection of training)
183
71.0
112.0
ratio of males in testData:
0.387978142077
ratio of males in training:
0.382768361582


In [311]:

data2=titanic_test.rdd.map(lambda x: (float(0.0),Vectors.dense([
                                                                      x.sexAsNum
                                                                     ,x.Pclass
                                                                     ,x.Age2
                                                                     ,x.Fare2
                                                                     ,x.isRev
                                                                     ,x.isMr
                                                                     ,x.isMrs
                                                                     ,x.isDr
                                                                     ,x.isMaster
                                                                     ,x.isMiss
                                                                     ,x.isArmy
                                                                     ,x.isRoyal
                                                                     ,x.embQ
                                                                     ,x.embS
                                                                     ,x.embC
                                                                     ,x.hasFam
                                                                     ,x.FareRelative
                                                                     ,x.hasParents
                                                                     ,x.isParent
                                                                     ,x.isHusband
                                                                     ,x.isWife                
                                                                     ,x.TicketNumber
                                                                     ,x.Deck
                                                                     ,x.cabA
                                                                     ,x.cabB
                                                                     ,x.cabC
                                                                     ,x.cabD
                                                                     ,x.cabE
                                                                     ,x.cabF
                                                                     ,x.hasCab
                                                                     ,x.nameLength
                                                                    ]))).toDF(["label","features"])

In [312]:
print model.stages

[StringIndexer_4c67bf5c7f5b478e46a4, VectorIndexer_47049ed3ea76c3ad916a, GBTClassificationModel (uid=GBTClassifier_4d14a51be68b8283fee1) with 10 trees]


In [316]:
uns_predictions = model.transform(data2)
uns_predictions.show(1000)
#uns_predictions.select("prediction", "indexedLabel", "features").show(5)
predsList=uns_predictions.rdd.map(lambda x:x.prediction).collect()
predictions = map(int, predsList)
passengerId= titanic_test.rdd.map(lambda x:x.PassengerId).collect()
answers=(zip(passengerId,predictions))

+-----+--------------------+------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|prediction|
+-----+--------------------+------------+--------------------+----------+
|  0.0|[1.0,3.0,34.5,7.8...|         0.0|[1.0,2.0,34.5,7.8...|       0.0|
|  0.0|[0.0,3.0,47.0,7.0...|         0.0|[0.0,2.0,47.0,7.0...|       0.0|
|  0.0|[1.0,2.0,62.0,9.6...|         0.0|[1.0,1.0,62.0,9.6...|       0.0|
|  0.0|[1.0,3.0,27.0,8.6...|         0.0|[1.0,2.0,27.0,8.6...|       0.0|
|  0.0|[0.0,3.0,22.0,12....|         0.0|[0.0,2.0,22.0,12....|       1.0|
|  0.0|[1.0,3.0,14.0,9.2...|         0.0|[1.0,2.0,14.0,9.2...|       0.0|
|  0.0|[0.0,3.0,30.0,7.6...|         0.0|[0.0,2.0,30.0,7.6...|       1.0|
|  0.0|[1.0,2.0,26.0,29....|         0.0|[1.0,1.0,26.0,29....|       0.0|
|  0.0|[0.0,3.0,18.0,7.2...|         0.0|[0.0,2.0,18.0,7.2...|       1.0|
|  0.0|[1.0,3.0,21.0,24....|         0.0|[1.0,2.0,21.0,24....|       0.0|
|  0.0|[1.0,3.0,28.72,7....|         0

In [317]:
outfile = open("/home/garth/titanic_answer_xgb.csv", "w")
print >> outfile, str("PassengerId,Survived")
print >> outfile, "\n".join(str(i).replace("(","").replace(")","").replace(" ","") for i in answers)
outfile.close()

## Neural Net (MLPC)

In [243]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [319]:
dataNN=features_df.rdd.map(lambda x: (float(x.Survived),Vectors.dense([
                                                                      x.sexAsNum
                                                                     ,x.Pclass
                                                                     ,x.Age2
                                                     #                ,x.Fare2
                                                     #                ,x.isRev
                                                     #                ,x.isMr
                                                     #                ,x.isMrs
                                                     #                ,x.isDr
                                                     #                ,x.isMaster
                                                     #                ,x.isMiss
                                                     #                ,x.isArmy
                                                     #                ,x.isRoyal
                                                     #                ,x.embQ
                                                     #                ,x.embS
                                                     #                ,x.embC
                                                                     ,x.hasFam
                                                     #                ,x.FareRelative
                                                     #                ,x.hasParents
                                                     #                ,x.isParent
                                                     #                ,x.isHusband
                                                     #                ,x.isWife                
                                                     #                ,x.TicketNumber
                                                     #                ,x.Deck
                                                     #                ,x.cabA
                                                     #                ,x.cabB
                                                     #                ,x.cabC
                                                     #                ,x.cabD
                                                     #                ,x.cabE
                                                     #                ,x.cabF
                                                     #                ,x.hasCab
                                                                     ,x.nameLength
                                                                    ]))).toDF(["label","features"])

dataTestNN=titanic_test.rdd.map(lambda x: (float(0.0),Vectors.dense([
                                                                      x.sexAsNum
                                                                     ,x.Pclass
                                                                     ,x.Age2
                                                      #               ,x.Fare2
                                                      #               ,x.isRev
                                                      #               ,x.isMr
                                                      #               ,x.isMrs
                                                      #               ,x.isDr
                                                      #               ,x.isMaster
                                                      #               ,x.isMiss
                                                     #                ,x.isArmy
                                                     #                ,x.isRoyal
                                                     #                ,x.embQ
                                                     #                ,x.embS
                                                     #                ,x.embC
                                                                     ,x.hasFam
                                                     #                ,x.FareRelative
                                                     #                ,x.hasParents
                                                     #                ,x.isParent
                                                     #                ,x.isHusband
                                                     #                ,x.isWife                
                                                     #                ,x.TicketNumber
                                                     #                ,x.Deck
                                                     #                ,x.cabA
                                                     #                ,x.cabB
                                                     #                ,x.cabC
                                                     #                ,x.cabD
                                                     #                ,x.cabE
                                                     #                ,x.cabF
                                                     #                ,x.hasCab
                                                                     ,x.nameLength
                                                                    ]))).toDF(["label","features"])

In [325]:
print dataNN.show(truncate=False)

print "number of features:"
print dataNN.count()
# Split the data into train and test
splits = dataNN.randomSplit([0.75, 0.25], 1234)
train = splits[0]
test = splits[1]


#test.show(1,truncate=False)
print "features length"
print (len(test.first()['features']))
# Make sure that first layer size is the same as the number of features! and last is the number of classes
layers = [5,16,16,2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=1000, layers=layers, blockSize=128, seed=114)

# train the model
model = trainer.fit(train)

print model

# compute accuracy on the test set
result = model.transform(test)
print result.count()
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: " + str(evaluator.evaluate(predictionAndLabels)))

+-----+------------------------+
|label|features                |
+-----+------------------------+
|0.0  |[1.0,3.0,22.0,1.0,23.0] |
|1.0  |[0.0,1.0,38.0,1.0,51.0] |
|1.0  |[0.0,3.0,26.0,0.0,22.0] |
|1.0  |[0.0,1.0,35.0,1.0,44.0] |
|0.0  |[1.0,3.0,35.0,0.0,24.0] |
|0.0  |[1.0,3.0,28.72,0.0,16.0]|
|0.0  |[1.0,1.0,54.0,0.0,23.0] |
|0.0  |[1.0,3.0,2.0,1.0,30.0]  |
|1.0  |[0.0,3.0,27.0,1.0,49.0] |
|1.0  |[0.0,2.0,14.0,1.0,35.0] |
|1.0  |[0.0,3.0,4.0,1.0,31.0]  |
|1.0  |[0.0,1.0,58.0,0.0,24.0] |
|0.0  |[1.0,3.0,20.0,0.0,30.0] |
|0.0  |[1.0,3.0,39.0,1.0,27.0] |
|0.0  |[0.0,3.0,14.0,0.0,36.0] |
|1.0  |[0.0,2.0,55.0,0.0,32.0] |
|0.0  |[1.0,3.0,2.0,1.0,20.0]  |
|1.0  |[1.0,2.0,32.77,0.0,28.0]|
|0.0  |[0.0,3.0,31.0,1.0,55.0] |
|1.0  |[0.0,3.0,33.51,0.0,23.0]|
+-----+------------------------+
only showing top 20 rows

None
number of features:
891
features length
5
MultilayerPerceptronClassifier_45e7ba1bba058bae3eb1
205
Accuracy: 0.8


In [139]:
#print float(result.filter("prediction = '0.0' and label ='0'").count()+result.filter("prediction = '1.0' and label ='1'").count())/(result.count())

In [326]:
uns_predictions = model.transform(dataTestNN)
uns_predictions.show(1)
#uns_predictions.select("prediction", "indexedLabel", "features").show(5)
predsList=uns_predictions.rdd.map(lambda x:x.prediction).collect()
predictions = map(int, predsList)
passengerId= titanic_test.rdd.map(lambda x:x.PassengerId).collect()
answers=(zip(passengerId,predictions))

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|[1.0,3.0,34.5,0.0...|       0.0|
+-----+--------------------+----------+
only showing top 1 row



In [327]:
outfile = open("/home/garth/titanic_answer_NN.csv", "w")
print >> outfile, str("PassengerId,Survived")
print >> outfile, "\n".join(str(i).replace("(","").replace(")","").replace(" ","") for i in answers)
outfile.close()

## Comparison between Training Prediction and Labels

In [140]:
print "If these are very different then something is wrong!"

print "__________________________________________"

ActualPercSurv = float(result.filter("label=1.0").count())/(result.count())
print "Survival Ratio of train_test:"
print ActualPercSurv

print "__________________________________________"

PredictedPercSurv = float(result.filter("prediction=1.0").count())/(result.count())
print "Predicted Survival Ratio:"
print PredictedPercSurv

print "__________________________________________"


print "Survival Ration of whole Training set (should be close to Survival Ratio of train_test):"
CompletePercSurv = float(dataNN.filter("label=1.0").count())/dataNN.count()
print CompletePercSurv

print "__________________________________________"

print "Survival Ratio of whole Train_train:"
CompletePercSurv = float(train.filter("label=1.0").count())/train.count()
print CompletePercSurv

If these are very different then something is wrong!
__________________________________________
Actual Survival Ratio of train_test:
0.357798165138
__________________________________________
Predicted Survival Ratio:
0.357798165138
__________________________________________
Survival Ration of whole Training set (should be close to Survival Ratio of train_test):
0.383838383838
__________________________________________
Survival Ratio of whole Train_train:
0.392273402675


## Comparison between Testing and Training Data Sets

In [170]:
print "__________________________________________"
TrainMaleRatio = float(titanic_train.filter("Sex='male'").count())/(titanic_train.count())
print "Survival Ratio of training set:"
print TrainMaleRatio
print "__________________________________________"

TestMaleRatio = float(titanic_test.filter("Sex='male'").count())/(titanic_test.count())
print "Survival Ratio of testing set:"
print TestMaleRatio
print "____________________________________________________________________________________"


Train1ClassRatio = float(titanic_train.filter("Pclass='1'").count())/(titanic_train.count())
print "1st Class Ratio of training set:"
print TrainMaleRatio
print "__________________________________________"

Test1ClassRatio = float(titanic_test.filter("Pclass='1'").count())/(titanic_test.count())
print "1st Class Ratio of testing set:"
print TestMaleRatio
print "____________________________________________________________________________________"

Train2ClassRatio = float(titanic_train.filter("Pclass='2'").count())/(titanic_train.count())
print "2nd Class Ratio of training set:"
print TrainMaleRatio
print "__________________________________________"

Test2ClassRatio = float(titanic_test.filter("Pclass='2'").count())/(titanic_test.count())
print "2nd Class Ratio of testing set:"
print TestMaleRatio
print "____________________________________________________________________________________"


TrainAvgAge = float(features_df.rdd.map(lambda x:x.Age2).reduce(lambda x,y:x+y))/(features_df.count())
print "Average age of training set:"
print TrainAvgAge
print "__________________________________________"

TestAvgAge = float(titanic_test.rdd.map(lambda x:float(x.Age2)).reduce(lambda x,y:x+y))/(titanic_test.count())
print "Average age of testing set:"
print TestAvgAge
print "____________________________________________________________________________________"

TrainAvgAge = float(features_df.rdd.map(lambda x:x.Age2).reduce(lambda x,y:x+y))/(features_df.count())
print "Average age of training set:"
print TrainAvgAge
print "__________________________________________"

TestAvgAge = float(titanic_test.rdd.map(lambda x:float(x.Age2)).reduce(lambda x,y:x+y))/(titanic_test.count())
print "Average age of testing set:"
print TestAvgAge
print "____________________________________________________________________________________"


__________________________________________
Survival Ratio of training set:
0.64758698092
__________________________________________
Survival Ratio of testing set:
0.636363636364
____________________________________________________________________________________
1st Class Ratio of training set:
0.64758698092
__________________________________________
1st Class Ratio of testing set:
0.636363636364
____________________________________________________________________________________
2nd Class Ratio of training set:
0.64758698092
__________________________________________
2nd Class Ratio of testing set:
0.636363636364
____________________________________________________________________________________
Average age of training set:
29.4242648709
__________________________________________
Average age of testing set:
29.7149521531
____________________________________________________________________________________
Average age of training set:
29.4242648709
_____________________________________